In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("SparkOps").getOrCreate()

df = spark.read.option("header", True).csv("/data/employees.csv")

df.show()


26/01/25 09:13:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+------+-------+-------+---------+------+-----+----------+----------------+
|emp_id|   name|   dept|     city|salary|bonus| join_date|          skills|
+------+-------+-------+---------+------+-----+----------+----------------+
|     1|  Alice|     IT|Bangalore| 70000| 5000|2020-01-15|    Python|Spark|
|     2|    Bob|     HR|    Delhi| 50000| NULL|2019-03-10|     Recruitment|
|     3|Charlie|     IT|Bangalore| 80000| 7000|2021-06-01|      Python|SQL|
|     4|  David|Finance|   Mumbai| 60000| 3000|2018-11-20|Accounting|Excel|
|     5|    Eva|     HR|    Delhi|  NULL| 2000|2022-02-25|   Communication|
|     6|  Frank|     IT|  Chennai| 75000| 6000|2019-07-18|      Java|Spark|
+------+-------+-------+---------+------+-----+----------+----------------+



🟢 Narrow Transformations (NO shuffle)

These operate row-by-row or within the same partition.

Column operations

select

withColumn

drop

alias

cast

when

explode ⚠️ (still narrow, but increases rows)

coalesce (when reducing partitions only)

Date functions (to_date, year, month, etc.)

Filtering

where

filter

Handling Nulls

dropna

fillna

select

In [6]:
df.select('name','salary').show()

+-------+------+
|   name|salary|
+-------+------+
|  Alice| 70000|
|    Bob| 50000|
|Charlie| 80000|
|  David| 60000|
|    Eva|  NULL|
|  Frank| 75000|
+-------+------+



withColumn = create or modify column

In [7]:
df.show()

+------+-------+-------+---------+------+-----+----------+----------------+
|emp_id|   name|   dept|     city|salary|bonus| join_date|          skills|
+------+-------+-------+---------+------+-----+----------+----------------+
|     1|  Alice|     IT|Bangalore| 70000| 5000|2020-01-15|    Python|Spark|
|     2|    Bob|     HR|    Delhi| 50000| NULL|2019-03-10|     Recruitment|
|     3|Charlie|     IT|Bangalore| 80000| 7000|2021-06-01|      Python|SQL|
|     4|  David|Finance|   Mumbai| 60000| 3000|2018-11-20|Accounting|Excel|
|     5|    Eva|     HR|    Delhi|  NULL| 2000|2022-02-25|   Communication|
|     6|  Frank|     IT|  Chennai| 75000| 6000|2019-07-18|      Java|Spark|
+------+-------+-------+---------+------+-----+----------+----------------+



In [9]:
df_withColumn = df.withColumn("new salary", df.salary+5000)

In [10]:
df_withColumn.show() 
#create

+------+-------+-------+---------+------+-----+----------+----------------+----------+
|emp_id|   name|   dept|     city|salary|bonus| join_date|          skills|new salary|
+------+-------+-------+---------+------+-----+----------+----------------+----------+
|     1|  Alice|     IT|Bangalore| 70000| 5000|2020-01-15|    Python|Spark|   75000.0|
|     2|    Bob|     HR|    Delhi| 50000| NULL|2019-03-10|     Recruitment|   55000.0|
|     3|Charlie|     IT|Bangalore| 80000| 7000|2021-06-01|      Python|SQL|   85000.0|
|     4|  David|Finance|   Mumbai| 60000| 3000|2018-11-20|Accounting|Excel|   65000.0|
|     5|    Eva|     HR|    Delhi|  NULL| 2000|2022-02-25|   Communication|      NULL|
|     6|  Frank|     IT|  Chennai| 75000| 6000|2019-07-18|      Java|Spark|   80000.0|
+------+-------+-------+---------+------+-----+----------+----------------+----------+



In [10]:
df_withColumn2 = df.withColumn('salary',col('salary')+col('bonus'))
df_withColumn2.show()
#modify

+------+-------+-------+---------+-------+-----+----------+----------------+
|emp_id|   name|   dept|     city| salary|bonus| join_date|          skills|
+------+-------+-------+---------+-------+-----+----------+----------------+
|     1|  Alice|     IT|Bangalore|75000.0| 5000|2020-01-15|    Python|Spark|
|     2|    Bob|     HR|    Delhi|   NULL| NULL|2019-03-10|     Recruitment|
|     3|Charlie|     IT|Bangalore|87000.0| 7000|2021-06-01|      Python|SQL|
|     4|  David|Finance|   Mumbai|63000.0| 3000|2018-11-20|Accounting|Excel|
|     5|    Eva|     HR|    Delhi|   NULL| 2000|2022-02-25|   Communication|
|     6|  Frank|     IT|  Chennai|81000.0| 6000|2019-07-18|      Java|Spark|
+------+-------+-------+---------+-------+-----+----------+----------------+



In [63]:
sc=spark.sparkContext.uiWebUrl

In [64]:
print(sc)

http://localhost:4041


In [8]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("DAG-UI-Test")
    .config("spark.ui.enabled", "true")
    .config("spark.ui.port", "4041")          # lock to 4041
    .config("spark.port.maxRetries", "0")
    .config("spark.driver.bindAddress", "0.0.0.0")
    .config("spark.driver.host", "localhost")
    .getOrCreate()
)

print("DAG UI URL:", spark.sparkContext.uiWebUrl)

DAG UI URL: http://localhost:4041


26/01/24 17:23:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [11]:
df.show()

+------+-------+-------+---------+------+-----+----------+----------------+
|emp_id|   name|   dept|     city|salary|bonus| join_date|          skills|
+------+-------+-------+---------+------+-----+----------+----------------+
|     1|  Alice|     IT|Bangalore| 70000| 5000|2020-01-15|    Python|Spark|
|     2|    Bob|     HR|    Delhi| 50000| NULL|2019-03-10|     Recruitment|
|     3|Charlie|     IT|Bangalore| 80000| 7000|2021-06-01|      Python|SQL|
|     4|  David|Finance|   Mumbai| 60000| 3000|2018-11-20|Accounting|Excel|
|     5|    Eva|     HR|    Delhi|  NULL| 2000|2022-02-25|   Communication|
|     6|  Frank|     IT|  Chennai| 75000| 6000|2019-07-18|      Java|Spark|
+------+-------+-------+---------+------+-----+----------+----------------+



drop - drop column 

In [13]:
df_drop=df.drop('skills')
df_drop.show()

+------+-------+-------+---------+------+-----+----------+
|emp_id|   name|   dept|     city|salary|bonus| join_date|
+------+-------+-------+---------+------+-----+----------+
|     1|  Alice|     IT|Bangalore| 70000| 5000|2020-01-15|
|     2|    Bob|     HR|    Delhi| 50000| NULL|2019-03-10|
|     3|Charlie|     IT|Bangalore| 80000| 7000|2021-06-01|
|     4|  David|Finance|   Mumbai| 60000| 3000|2018-11-20|
|     5|    Eva|     HR|    Delhi|  NULL| 2000|2022-02-25|
|     6|  Frank|     IT|  Chennai| 75000| 6000|2019-07-18|
+------+-------+-------+---------+------+-----+----------+



alias - rename column 

In [15]:
df_alias=df.select(col('dept').alias('department'))
df_alias.show()

+----------+
|department|
+----------+
|        IT|
|        HR|
|        IT|
|   Finance|
|        HR|
|        IT|
+----------+



cast - change data type 

In [24]:
df_cast=df.withColumn('dept', col('dept').cast(IntegerType()))

In [22]:
df_cast.show()

+------+-------+-----+---------+------+-----+----------+----------------+
|emp_id|   name| dept|     city|salary|bonus| join_date|          skills|
+------+-------+-----+---------+------+-----+----------+----------------+
|     1|  Alice|70000|Bangalore| 70000| 5000|2020-01-15|    Python|Spark|
|     2|    Bob|50000|    Delhi| 50000| NULL|2019-03-10|     Recruitment|
|     3|Charlie|80000|Bangalore| 80000| 7000|2021-06-01|      Python|SQL|
|     4|  David|60000|   Mumbai| 60000| 3000|2018-11-20|Accounting|Excel|
|     5|    Eva| NULL|    Delhi|  NULL| 2000|2022-02-25|   Communication|
|     6|  Frank|75000|  Chennai| 75000| 6000|2019-07-18|      Java|Spark|
+------+-------+-----+---------+------+-----+----------+----------------+



In [19]:
df_cast.schema['salary'].datatype

AttributeError: 'StructField' object has no attribute 'datatype'

In [23]:
df_cast.printSchema()


root
 |-- emp_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- dept: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- bonus: string (nullable = true)
 |-- join_date: string (nullable = true)
 |-- skills: string (nullable = true)



In [25]:
df_cast=df.withColumn('salary', col('salary').cast(IntegerType()))

In [26]:
df_cast.show()

+------+-------+-------+---------+------+-----+----------+----------------+
|emp_id|   name|   dept|     city|salary|bonus| join_date|          skills|
+------+-------+-------+---------+------+-----+----------+----------------+
|     1|  Alice|     IT|Bangalore| 70000| 5000|2020-01-15|    Python|Spark|
|     2|    Bob|     HR|    Delhi| 50000| NULL|2019-03-10|     Recruitment|
|     3|Charlie|     IT|Bangalore| 80000| 7000|2021-06-01|      Python|SQL|
|     4|  David|Finance|   Mumbai| 60000| 3000|2018-11-20|Accounting|Excel|
|     5|    Eva|     HR|    Delhi|  NULL| 2000|2022-02-25|   Communication|
|     6|  Frank|     IT|  Chennai| 75000| 6000|2019-07-18|      Java|Spark|
+------+-------+-------+---------+------+-----+----------+----------------+



when - work like case 

In [28]:
df_when = df.withColumn(
           'grade',
            when(col('salary')>=56000 ,'Mid')
            .when(col('salary')>=70000,'High')
            .otherwise('Low')


)
df_when.show()

+------+-------+-------+---------+------+-----+----------+----------------+-----+
|emp_id|   name|   dept|     city|salary|bonus| join_date|          skills|grade|
+------+-------+-------+---------+------+-----+----------+----------------+-----+
|     1|  Alice|     IT|Bangalore| 70000| 5000|2020-01-15|    Python|Spark|  Mid|
|     2|    Bob|     HR|    Delhi| 50000| NULL|2019-03-10|     Recruitment|  Low|
|     3|Charlie|     IT|Bangalore| 80000| 7000|2021-06-01|      Python|SQL|  Mid|
|     4|  David|Finance|   Mumbai| 60000| 3000|2018-11-20|Accounting|Excel|  Mid|
|     5|    Eva|     HR|    Delhi|  NULL| 2000|2022-02-25|   Communication|  Low|
|     6|  Frank|     IT|  Chennai| 75000| 6000|2019-07-18|      Java|Spark|  Mid|
+------+-------+-------+---------+------+-----+----------+----------------+-----+



In [29]:
df.show()

+------+-------+-------+---------+------+-----+----------+----------------+
|emp_id|   name|   dept|     city|salary|bonus| join_date|          skills|
+------+-------+-------+---------+------+-----+----------+----------------+
|     1|  Alice|     IT|Bangalore| 70000| 5000|2020-01-15|    Python|Spark|
|     2|    Bob|     HR|    Delhi| 50000| NULL|2019-03-10|     Recruitment|
|     3|Charlie|     IT|Bangalore| 80000| 7000|2021-06-01|      Python|SQL|
|     4|  David|Finance|   Mumbai| 60000| 3000|2018-11-20|Accounting|Excel|
|     5|    Eva|     HR|    Delhi|  NULL| 2000|2022-02-25|   Communication|
|     6|  Frank|     IT|  Chennai| 75000| 6000|2019-07-18|      Java|Spark|
+------+-------+-------+---------+------+-----+----------+----------------+



In [40]:
df.printSchema()

root
 |-- emp_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- city: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- bonus: string (nullable = true)
 |-- join_date: string (nullable = true)
 |-- skills: string (nullable = true)



In [37]:
from pyspark.sql.functions import explode


In [38]:
df.show()

+------+-------+-------+---------+------+-----+----------+----------------+
|emp_id|   name|   dept|     city|salary|bonus| join_date|          skills|
+------+-------+-------+---------+------+-----+----------+----------------+
|     1|  Alice|     IT|Bangalore| 70000| 5000|2020-01-15|    Python|Spark|
|     2|    Bob|     HR|    Delhi| 50000| NULL|2019-03-10|     Recruitment|
|     3|Charlie|     IT|Bangalore| 80000| 7000|2021-06-01|      Python|SQL|
|     4|  David|Finance|   Mumbai| 60000| 3000|2018-11-20|Accounting|Excel|
|     5|    Eva|     HR|    Delhi|  NULL| 2000|2022-02-25|   Communication|
|     6|  Frank|     IT|  Chennai| 75000| 6000|2019-07-18|      Java|Spark|
+------+-------+-------+---------+------+-----+----------+----------------+



In [39]:
df_explode= df.withColumn('skills',explode('skills'))
df_explode.show()

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "explode(skills)" due to data type mismatch: Parameter 1 requires the ("ARRAY" or "MAP") type, however "skills" has the type "STRING".;
'Project [emp_id#17, name#18, dept#19, city#20, salary#21, bonus#22, join_date#23, explode(skills#24) AS skills#589]
+- Relation [emp_id#17,name#18,dept#19,city#20,salary#21,bonus#22,join_date#23,skills#24] csv


skills: string (nullable = true)
explode will NOT work on a string
It only works on array or map columns.

In [41]:
from pyspark.sql.functions import split

In [47]:
df_explode= df.withColumn('skills',explode(split('skills',"|")))
df_explode.show()

+------+-----+----+---------+------+-----+----------+------+
|emp_id| name|dept|     city|salary|bonus| join_date|skills|
+------+-----+----+---------+------+-----+----------+------+
|     1|Alice|  IT|Bangalore| 70000| 5000|2020-01-15|     P|
|     1|Alice|  IT|Bangalore| 70000| 5000|2020-01-15|     y|
|     1|Alice|  IT|Bangalore| 70000| 5000|2020-01-15|     t|
|     1|Alice|  IT|Bangalore| 70000| 5000|2020-01-15|     h|
|     1|Alice|  IT|Bangalore| 70000| 5000|2020-01-15|     o|
|     1|Alice|  IT|Bangalore| 70000| 5000|2020-01-15|     n|
|     1|Alice|  IT|Bangalore| 70000| 5000|2020-01-15|     ||
|     1|Alice|  IT|Bangalore| 70000| 5000|2020-01-15|     S|
|     1|Alice|  IT|Bangalore| 70000| 5000|2020-01-15|     p|
|     1|Alice|  IT|Bangalore| 70000| 5000|2020-01-15|     a|
|     1|Alice|  IT|Bangalore| 70000| 5000|2020-01-15|     r|
|     1|Alice|  IT|Bangalore| 70000| 5000|2020-01-15|     k|
|     1|Alice|  IT|Bangalore| 70000| 5000|2020-01-15|      |
|     2|  Bob|  HR|    D

In [48]:
df_explode= df.withColumn('skills',explode(split('skills',"\\|")))
df_explode.show()

+------+-------+-------+---------+------+-----+----------+-------------+
|emp_id|   name|   dept|     city|salary|bonus| join_date|       skills|
+------+-------+-------+---------+------+-----+----------+-------------+
|     1|  Alice|     IT|Bangalore| 70000| 5000|2020-01-15|       Python|
|     1|  Alice|     IT|Bangalore| 70000| 5000|2020-01-15|        Spark|
|     2|    Bob|     HR|    Delhi| 50000| NULL|2019-03-10|  Recruitment|
|     3|Charlie|     IT|Bangalore| 80000| 7000|2021-06-01|       Python|
|     3|Charlie|     IT|Bangalore| 80000| 7000|2021-06-01|          SQL|
|     4|  David|Finance|   Mumbai| 60000| 3000|2018-11-20|   Accounting|
|     4|  David|Finance|   Mumbai| 60000| 3000|2018-11-20|        Excel|
|     5|    Eva|     HR|    Delhi|  NULL| 2000|2022-02-25|Communication|
|     6|  Frank|     IT|  Chennai| 75000| 6000|2019-07-18|         Java|
|     6|  Frank|     IT|  Chennai| 75000| 6000|2019-07-18|        Spark|
+------+-------+-------+---------+------+-----+----

coalesce - if null value in column then it will change null to value

In [51]:
df_coalesce = df.withColumn('salary', coalesce(col('salary'),lit(0)))
df_coalesce.show()                           

+------+-------+-------+---------+------+-----+----------+----------------+
|emp_id|   name|   dept|     city|salary|bonus| join_date|          skills|
+------+-------+-------+---------+------+-----+----------+----------------+
|     1|  Alice|     IT|Bangalore| 70000| 5000|2020-01-15|    Python|Spark|
|     2|    Bob|     HR|    Delhi| 50000| NULL|2019-03-10|     Recruitment|
|     3|Charlie|     IT|Bangalore| 80000| 7000|2021-06-01|      Python|SQL|
|     4|  David|Finance|   Mumbai| 60000| 3000|2018-11-20|Accounting|Excel|
|     5|    Eva|     HR|    Delhi|     0| 2000|2022-02-25|   Communication|
|     6|  Frank|     IT|  Chennai| 75000| 6000|2019-07-18|      Java|Spark|
+------+-------+-------+---------+------+-----+----------+----------------+



date  - extract date 

In [52]:
df.withColumn("join_year", year("join_date")) \
  .select("name","join_year").show()


+-------+---------+
|   name|join_year|
+-------+---------+
|  Alice|     2020|
|    Bob|     2019|
|Charlie|     2021|
|  David|     2018|
|    Eva|     2022|
|  Frank|     2019|
+-------+---------+



In [62]:
df_date = df.withColumn('join_year',year('join_date')) \
            .select('name','join_year')
df_date.show()

+-------+---------+
|   name|join_year|
+-------+---------+
|  Alice|     2020|
|    Bob|     2019|
|Charlie|     2021|
|  David|     2018|
|    Eva|     2022|
|  Frank|     2019|
+-------+---------+



In [65]:
from pyspark.sql.functions import year, month, dayofmonth, to_date, col


date function -  extraction 
year
month
date 

In [67]:
df_year = df.withColumn('join_year', year('join_date')) \
        .select('join_year')
df_year.show()

+---------+
|join_year|
+---------+
|     2020|
|     2019|
|     2021|
|     2018|
|     2022|
|     2019|
+---------+



In [68]:
df_month = df.withColumn('join_month', month('join_date'))\
            .select('join_month')
df_month.show()

+----------+
|join_month|
+----------+
|         1|
|         3|
|         6|
|        11|
|         2|
|         7|
+----------+



In [70]:
df_month = df.withColumn('join_date', dayofmonth('join_date'))\
            .select('join_date')
df_month.show()

+---------+
|join_date|
+---------+
|       15|
|       10|
|        1|
|       20|
|       25|
|       18|
+---------+



where / filter

In [71]:
df.show()

+------+-------+-------+---------+------+-----+----------+----------------+
|emp_id|   name|   dept|     city|salary|bonus| join_date|          skills|
+------+-------+-------+---------+------+-----+----------+----------------+
|     1|  Alice|     IT|Bangalore| 70000| 5000|2020-01-15|    Python|Spark|
|     2|    Bob|     HR|    Delhi| 50000| NULL|2019-03-10|     Recruitment|
|     3|Charlie|     IT|Bangalore| 80000| 7000|2021-06-01|      Python|SQL|
|     4|  David|Finance|   Mumbai| 60000| 3000|2018-11-20|Accounting|Excel|
|     5|    Eva|     HR|    Delhi|  NULL| 2000|2022-02-25|   Communication|
|     6|  Frank|     IT|  Chennai| 75000| 6000|2019-07-18|      Java|Spark|
+------+-------+-------+---------+------+-----+----------+----------------+



In [72]:
df_select = df.filter(col('city')=='Delhi')
df_select.show()

+------+----+----+-----+------+-----+----------+-------------+
|emp_id|name|dept| city|salary|bonus| join_date|       skills|
+------+----+----+-----+------+-----+----------+-------------+
|     2| Bob|  HR|Delhi| 50000| NULL|2019-03-10|  Recruitment|
|     5| Eva|  HR|Delhi|  NULL| 2000|2022-02-25|Communication|
+------+----+----+-----+------+-----+----------+-------------+

